In [95]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2

# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


# Step 2 - Preparing the train/test data and training the model

# Code copied from - https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data/train',
                                                 target_size=(64, 64),
                                                 batch_size=5,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data/test',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
classifier.fit_generator(
        training_set,
        steps_per_epoch=600, # No of images in training set
        epochs=10,
        validation_data=test_set,
        validation_steps=30)# No of images in test set


# Saving the model
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
classifier.save_weights('model-bw.h5')

Found 600 images belonging to 6 classes.
Found 30 images belonging to 6 classes.
Epoch 1/10
600/600 [==============================] - 27s 45ms/step - loss: 0.6654 - accuracy: 0.7367 - val_loss: 0.3556 - val_accuracy: 0.9000
Epoch 2/10
600/600 [==============================] - 26s 44ms/step - loss: 0.1772 - accuracy: 0.9410 - val_loss: 0.0615 - val_accuracy: 0.9333
Epoch 3/10
600/600 [==============================] - 26s 43ms/step - loss: 0.0691 - accuracy: 0.9753 - val_loss: 0.0060 - val_accuracy: 0.9000
Epoch 4/10
600/600 [==============================] - 26s 44ms/step - loss: 0.0467 - accuracy: 0.9800 - val_loss: 0.0544 - val_accuracy: 0.9667
Epoch 5/10
600/600 [==============================] - 25s 41ms/step - loss: 0.0418 - accuracy: 0.9880 - val_loss: 7.0036e-05 - val_accuracy: 0.9667
Epoch 6/10
600/600 [==============================] - 26s 43ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 5.6830e-04 - val_accuracy: 0.9333
Epoch 7/10
600/600 [===========================

In [113]:
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os
from skimage.transform import resize

# Loading the model
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")
roi=cv2.imread("two2.jpeg")
def detect(frame):
    try:
        img = resize(frame,(64,64,1))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = classifier.predict(img)
        prediction = classifier.predict_classes(img)
        return prediction
    except AttributeError:
        print("shape not found")
data=detect(roi)

Loaded model from disk


In [114]:
data

array([2], dtype=int64)